### doc_topics
ESTABELECE RELAÇÕES ENTRE DOCUMENTOS E TÓPICOS NA BASE DE DADOS NO MYSQL. TRABALHAMOS COM O ACERVO **ANTONIO AZEREDO DA SILVEIRA, MINISTÉRIO DAS RELAÇÕES EXTERIORES**.

### Observações

#### Sobre como cria tabela CSV com limite documentos por tópico

In [1]:
import nltk
import os
import codecs
import string
import matplotlib.pyplot as plt
import matplotlib as mpl
from gensim import corpora, models, similarities #Latent Dirichlet Allocation implementation with Gensim
import pyLDAvis
import pyLDAvis.gensim
from IPython.display import clear_output

import getpass
from sshtunnel import SSHTunnelForwarder
import pymysql

import pickle

Couldn't import dot_parser, loading of dot files will not be possible.


Using gpu device 0: Quadro K4200 (CNMeM is disabled, cuDNN 5103)
/usr/local/lib/python3.5/dist-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
/usr/local/lib/python3.5/dist-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
/usr/local/lib/python3.5/dist-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
/usr/local/lib/python3.5/dist-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
/usr/local/lib/python3.5/dist-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is depreca

In [2]:
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
Verifica qual sistema operacional está sendo usado e cria variáveis. Renato = Linux ; Marcelo = nt (Windows)
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
if os.name == 'nt':
    encoding_type = 'utf-8'
    ssh_priv_key = 'C:/Users/marcelo.ribeiro/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Remoto/marcelo_priv_rsa'
    ssh_user='marcelobribeiro'
    sql_user='marcelobribeiro'
    outputs = 'C:/Users/marcelo.ribeiro/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Azeredo Papers/Processamento/text-learning-tools/outputs/'
    file_lda = 'C:/Users/marcelo.ribeiro/Desktop/pickles/model_lda_100_rs_00.pkl'
else:
    encoding_type = 'ISO-8859-1'
    ssh_priv_key = '/home/rsouza/.ssh/id_rsa'
    ssh_user='rsouza'
    sql_user='rsouza'
    outputs = '../outputs/'
    file_lda = outputs+'model_lda_100_rs_00.pkl'

#### Load pickle files

In [3]:

file_corpus = outputs+'LDAcorpus.pkl'
file_dictionary = outputs+'LDAdictionary.pkl'

corpus = pickle.load(open(file_corpus, 'rb'))
dictionary = pickle.load(open(file_dictionary, 'rb'))
lda = pickle.load(open(file_lda, 'rb'))

In [4]:
len(corpus)

9058

In [5]:
lda.print_topics(-1, num_words=5)

[(0,
  '0.051*excelência + 0.034*senhor + 0.028*exteriores + 0.025*relações + 0.023*ministério'),
 (1, '0.023*torres + 0.022*broissa + 0.017*anglo + 0.017*egp + 0.017*arlette'),
 (2,
  '0.065*chanceler + 0.032*exteriores + 0.031*silveira + 0.031*brasil + 0.018*ministro'),
 (3,
  '0.037*presidente + 0.036*brasil + 0.015*geisel + 0.014*disse + 0.008*ter'),
 (4, '0.035*cuba + 0.027*venezuela + 0.027*oea + 0.026*reunião + 0.024*chile'),
 (5,
  '0.056*senhor + 0.034*excelência + 0.027*leurs + 0.019*senhora + 0.019*place'),
 (6, '0.009*ar + 0.007*ro + 0.007*ão + 0.006*cc + 0.006*ea'),
 (7,
  '0.085*nuclear + 0.036*acordo + 0.035*energia + 0.030*nucleares + 0.023*brasil'),
 (8, '0.038*cl + 0.031*ps + 0.019*uh + 0.015*hr + 0.012*ic'),
 (9,
  '0.075*silveira + 0.062*azeredo + 0.054*antonio + 0.045*ministro + 0.026*exteriores'),
 (10,
  '0.145*instituto + 0.127*universidade + 0.032*ciências + 0.030*professores + 0.028*faculdade'),
 (11, '0.024*buenos + 0.023*aires + 0.017*brasil + 0.016*série + 

In [6]:
# Caso queira trabalhar apenas com os tópicos mais coesos
main_topics_list = [99, 61, 39, 69, 63, 19, 17, 82, 56, 42, 94, 18, 89, 4, 49, 7, 32, 25, 78, 15, 73, 45, 27, 16, 97, 47, 26, 
                   58, 59, 36, 10, 57, 86, 34, 28, 53, 12, 37, 51, 76, 83, 93, 46, 31, 11, 13, 65]

#### Gera a distribuição de tópicos para um novo documento
Alimenta a base mysql com dados de score (relação) entre tópicos e documentos, gerando a distribuição de tópicos para um novo documento.

Notas:
* Os documentos são selecionados da base SQL com alguns filtros: idioma português e legibilidade acima de 40%.
* 'pt' = língua portuguesa. 
* No caso de documentos que não conseguimos identificar a legibilidade, por serem curtos (menos de 10 frases totais), foram classificados com a numeração '-1'.

In [8]:
count = 0
pass_sshkey = getpass.getpass()
pass_mysql = getpass.getpass()
with SSHTunnelForwarder(('200.20.164.147', 22),
                        ssh_private_key = ssh_priv_key,
                        ssh_private_key_password = pass_sshkey,
                        ssh_username=ssh_user,
                        remote_bind_address=('127.0.0.1', 3306)) as server:
    
    conn = pymysql.connect(host='localhost', 
                           port=server.local_bind_port, 
                           user=sql_user,
                           passwd=pass_mysql,
                           db='CPDOC_AS',
                           use_unicode=True, 
                           charset="utf8")
    cur = conn.cursor()
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    insere dados na base mysql
    captura documentos da tabela docs, transfere para a tabela topic-doc e insere dados de tópicos
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    
    cur.execute("DROP TABLE IF EXISTS topic_doc")
    cur.execute('''CREATE TABLE IF NOT EXISTS topic_doc
               (doc_id VARCHAR(31), topic_id smallint(6), topic_score FLOAT
               DEFAULT NULL)
               ENGINE=MyISAM DEFAULT CHARSET='utf8';''')

    cur.execute("SELECT * FROM CPDOC_AS.docs WHERE main_language = 'pt' AND (readability > 0.4 OR readability = -1) ")  
    data = cur.fetchall()
    numrows = cur.rowcount
    percentil = numrows/100
    
    for row in data:
        count += 1
        #if row is None: break
        
        ### mede percentual de conclusão da tarefa ###
        if count % 100 == 0: 
            clear_output()
            print(int(count/percentil),'% done')
        
        text =  row[4]
        text = text.split()
        text_bow = dictionary.doc2bow(text)
        score_list = lda[text_bow]
        doc_id = row[0]
        for score in score_list:
            topic_id = str(score[0])
            topic_score = str(score[1])
            #if topic_id not in main_topics_list: continue
            query = "INSERT INTO topic_doc VALUES (%s, %s, %s)"
            cur.execute(query, (doc_id, topic_id, topic_score))
            #print(doc_id, topic_id, topic_score)   
        cur.execute("ALTER TABLE CPDOC_AS.topic_doc ORDER BY topic_id ASC, topic_score DESC")

99 % done
